**Author** : Yash Gupta

**Description**: The below code will fetch all explore and views and save it as a CSV file in GDrive.

Instructions:


1.   Press **Ctrl+F9** to execute
2.   Allow colab access to your google drive.
3.   Wait for the beep sound before checking output ⏰.
4.   Output will get saved in your local GDrive as `explore_view_details.csv`



In [1]:
!pip install looker_sdk
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

     |████████████████████████████████| 336 kB 5.2 MB/s 
Mounted at /content/drive


In [2]:
import os
os.environ["LOOKERSDK_CLIENT_SECRET"] = "abc"
os.environ["LOOKERSDK_CLIENT_ID"] = "abc"
os.environ["LOOKERSDK_BASE_URL"] = "https://abc.looker.com:19999"
os.environ["LOOKERSDK_VERIFY_SSL"] = "True"

In [3]:
import looker_sdk
import pandas as pd
sdk = looker_sdk.init31()  # or init40() for v4.0 API
# my_user = sdk.me()
# # output can be treated like a dictionary
# print(my_user["first_name"])
# # or a model instance (User in this case)
# print(my_user.first_name)
# print(type(my_user))

all_models = sdk.all_lookml_models()

explore_dict = {"ExploreName":[],"ExploreLabel":[],"ModelName":[]};
explore_details_dict = {"ExploreName":[],"sql_table_name":[],"ScopeViewsList":[]};


model_df = pd.DataFrame(
      {'name': [x.name for x in all_models],
     'label': [x.label for x in all_models],
     'allowed_connections': [x.allowed_db_connection_names for x in all_models]
    })

for model in all_models:
  for explore in model.explores:
    explore_dict["ExploreName"].append(explore.name)
    explore_dict["ExploreLabel"].append(explore.label)
    explore_dict["ModelName"].append(model.name)

    explore_details = sdk.lookml_model_explore(lookml_model_name=model.name,explore_name=explore.name,fields="name,scopes,sql_table_name")
    explore_details_dict["ExploreName"].append(explore_details.name)
    explore_details_dict["sql_table_name"].append(explore_details.sql_table_name)
    explore_details_dict["ScopeViewsList"].append(explore_details.scopes)
    

explore_df = pd.DataFrame.from_dict(explore_dict)
explore_details_df =  pd.DataFrame.from_dict(explore_details_dict)
df_stack = pd.DataFrame(explore_details_df.ScopeViewsList.to_list(), index=explore_details_df.ExploreName).stack()
df_stack = df_stack.reset_index(["ExploreName"])
df_stack.columns = ["ExploreName", "ScopeViewList"]
explore_details_df =  pd.merge( df_stack,explore_details_df, how ='left', on ='ExploreName')
explore_details_df =  pd.merge( explore_details_df,explore_df, how ='left', on ='ExploreName')
explore_details_df =  pd.merge( explore_details_df,model_df, how ='left', left_on ='ModelName',right_on = 'name')


In [4]:
explore_details_df.to_csv('/content/drive/MyDrive/explore_view_details.csv', index=False,header=True)


In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')